In [ ]:
# All the libraries
!python -m pip install gwpy
!pip install astropy==4.2.1
!pip install astropy
!pip install nnAudio
!pip install tqdm

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import signal
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import tqdm
import gwpy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


In [ ]:
df = pd.read_csv("/kaggle/input/g2net-gravitational-wave-detection/training_labels.csv")
fnames = []
path = '/kaggle/input/g2net-gravitational-wave-detection/train/0/0/0'
pde=[]

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        fnames.append(os.path.join(dirname,filename))
        pde.append(filename)
len(fnames)
fnames[:20]

# Reading the NPY file


In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')
plt.figure(figsize= (15,10))

for i in range(3):
    image = np.load(fnames[1])
    ax = plt.subplot(3,1,i+1)
    image = image[i,:]/np.max(image[i,:])
    plt.plot(image)

In [ ]:
image = np.load(fnames[4])
image.shape
image[0]

# A function to Show all the waveforms contained in one Np.array

In [ ]:
def wavesinone(fno):
    """
    Function takes filenumber as input and plots all the 3 or 4 waves in that np.array
    """
    


    plt.style.use('ggplot')
    plt.figure(figsize= (15,10))

    for i in range(3):
        image = np.load(fnames[fno])
        ax = plt.subplot(3,1,i+1)
        image = image[i,:]/np.max(image[i,:])
        plt.plot(image)

In [ ]:
wavesinone(10)

# Writing a function to show the frequency response

In [ ]:
def freqinone(fno):
    b = np.load(fnames[fno])
    w, h = signal.freqz(b[0][:])

    import matplotlib.pyplot as plt

    fig, ax1 = plt.subplots(figsize= (17,4))
    ax1.set_title('LIGO Hanford Digital filter frequency response')

    ax1.plot(w, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Amplitude [dB]', color='b')
    ax1.set_xlabel('Frequency [rad/sample]')

    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    ax2.plot(w, angles, 'g')
    ax2.set_ylabel('Angle (radians)', color='g')
    ax2.grid()
    ax2.axis('tight')
    plt.show()

    b = np.load(fnames[10])
    w, h = signal.freqz(b[1][:])


    fig1,ax3=  plt.subplots(figsize= (17,4))
    ax3.set_title('LIGO Livingston Digital filter frequency response')

    ax3.plot(w, 20 * np.log10(abs(h)), 'b')
    ax3.set_ylabel('Amplitude [dB]', color='b')
    ax3.set_xlabel('Frequency [rad/sample]')

    ax4 = ax3.twinx()
    angles = np.unwrap(np.angle(h))
    ax4.plot(w, angles, 'g')
    ax4.set_ylabel('Angle (radians)', color='g')
    ax4.grid()
    ax4.axis('tight')
    plt.show()

    b = np.load(fnames[10])
    w, h = signal.freqz(b[2][:])


    fig2,ax5=  plt.subplots(figsize= (17,4))
    ax5.set_title('LIGO Livingston Digital filter frequency response')

    ax5.plot(w, 20 * np.log10(abs(h)), 'b')
    ax5.set_ylabel('Amplitude [dB]', color='b')
    ax5.set_xlabel('Frequency [rad/sample]')

    ax6 = ax5.twinx()
    angles = np.unwrap(np.angle(h))
    ax6.plot(w, angles, 'g')
    ax6.set_ylabel('Angle (radians)', color='g')
    ax6.grid()
    ax6.axis('tight')
    plt.show()

In [ ]:
freqinone(2)

# Create a Function to extract  the info from 3 sensors


In [ ]:
def tseries(fno):
    b = np.load(fnames[fno])
    t1= b[0][:]
    t2 = b[1][:]
    t3 = b[2][:]
    
    return t1,t2,t3

In [ ]:
a,b,c = tseries(1)

a.shape,b.shape,c.shape

# Create a Function to convert extracted Time series into tensors


In [ ]:
def totensors(fno):
    a,b,c = tseries(fno)
    t1 = tf.convert_to_tensor(a)
    t2 = tf.convert_to_tensor(b)
    t3 = tf.convert_to_tensor(c)
    
    return t1,t2,t3
    

In [ ]:
a,b,c = totensors(32)
d = tf.constant(a)
d.shape, a.shape

In [ ]:
model = keras.Sequential([
    layers.Dense(512 , activation = 'relu', input_shape=[3]),
    layers.Dense(1024 , activation = 'relu'),
    layers.Dense(512 , activation = 'relu'),
    layers.Dense(1),
    
])

model.compile(
    optimizer = "adam",
    loss = 'BinaryCrossentropy'
)

In [ ]:
df.head()

In [ ]:
pd= df.id
target = df.target
pd = pd+".npy"
pd

In [ ]:
t=[]
x=[]
for i in range(155):
    for j in range(560000):
        if pde[i] == pd[j]:
                x.append(j)
                t.append(target[j])
                print(j,len(x))
                
len(x)

In [ ]:
#new_pde = {x.replace(".npy","") for x in pde}
#new_pde
import pandas as pd
df1= pd.DataFrame(list(zip(pde,t)),columns= ["id","target"])
df1


# Function to create a data frame for every array and label


In [ ]:
df1.head()

In [ ]:
aa =[]
bb =[]
cc =[]

for i in range(155):
    a,b,c = tseries(i)
    aa.append(a)
    bb.append(b)
    cc.append(c)
    



In [ ]:
df11 = df1.target
df11

In [ ]:
df1 = df1.drop("target",axis = 1)
df1

In [ ]:
df1["hanford"]=aa
df1["living"]=bb
df1["virgo"]=cc

In [ ]:
df1["target"] = df11
df1.head()

In [ ]:
df1 = df1.drop("id",axis = 1)
df1

# Splitting data into training and test sets


In [ ]:
X_train,X_val,y_train,y_val = train_test_split(df1.drop("target",axis = 1),
                                               df1.target,test_size = 0.2,
                                               random_state=13)
len(X_train),len(X_val),len(y_train),len(y_val)

In [ ]:
X_train.head()

In [ ]:
def my_func(arg):
   
    arg = np.array(arg,dtype = "object") 
    print(arg.dtype)
    arg = tf.convert_to_tensor(arg, dtype=np.float64)
    return arg

In [ ]:
def my_func(arg):
    """
    function will take Fnames input and convert them into 
    """
    for i in range(124):

In [ ]:
d= X_train[1:2]
d = np.array(d,dtype = "object")
print(d.shape)
t = tf.convert_to_tensor(d, dtype=np.float64)
t

In [ ]:

X_train[1:2]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(),
          "Random Forest": RandomForestClassifier()}

# Create a function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of differetn Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    # Set random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_val,
                             y_train=y_train,
                             y_test=y_val)

model_scores